In [1]:
import os
import numpy
import csv
import cv2
from cv2 import aruco
import pickle
import glob
import shutil
import natsort
import matplotlib as mpl
import matplotlib.pyplot as plt
from tabulate import tabulate

In [2]:
# ChAruco board variables
CHARUCOBOARD_ROWCOUNT = 6
CHARUCOBOARD_COLCOUNT = 6 
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_6X6_50)

In [3]:
# Create constants to be passed into OpenCV and Aruco methods
CHARUCO_BOARD = aruco.CharucoBoard_create(
        squaresX=CHARUCOBOARD_COLCOUNT,
        squaresY=CHARUCOBOARD_ROWCOUNT,
        squareLength=0.5,
        markerLength=0.375,
        dictionary=ARUCO_DICT)

In [4]:
corners_all = [] # Corners discovered in all images processed
ids_all = [] # Aruco ids corresponding to corners discovered
image_size = None # Determined at runtime

In [5]:
# images = glob.glob('C:\\Users\\bidayelab\\Documents\\chArUco-detection\\06072022-hind specific\\10\\t1\\camC\\*.png')
# images = glob.glob('C:\\Users\\bidayelab\\Documents\\chArUco-detection\\06072022-hind specific\\10\\t2\\camC\\camC0000.png')
# path_name = 'C:\\Users\\bidayelab\\Documents\\chArUco-detection\\06102022\set1\A\*.png'
# path_name = 'C:\\Users\\bidayelab\\Documents\\chArUco-detection\\06102022\set1\A2\*.png'


# images = glob.glob('C:\\Users\\bidayelab\\Documents\\chArUco-detection\\06102022\set1\A\*.png')
# images = natsort.natsorted(images)
# images = glob.glob('C:\\Users\\bidayelab\\Documents\\chArUco-detection\\06102022\set1\A2\*.png')

In [6]:
# Destroy any open CV windows
cv2.destroyAllWindows()

In [7]:
# Loop through images glob'ed
def analyze_images(path, images):
    
    # ChAruco board variables
    CHARUCOBOARD_ROWCOUNT = 6
    CHARUCOBOARD_COLCOUNT = 6 
    ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_6X6_50)
    
    # Create constants to be passed into OpenCV and Aruco methods
    CHARUCO_BOARD = aruco.CharucoBoard_create(
            squaresX=CHARUCOBOARD_COLCOUNT,
            squaresY=CHARUCOBOARD_ROWCOUNT,
            squareLength=0.5,
            markerLength=0.375,
            dictionary=ARUCO_DICT)
    
    corners_all = [] # Corners discovered in all images processed
    ids_all = [] # Aruco ids corresponding to corners discovered
    image_size = None # Determined at runtime
    img_corners = []
    img_ids = []
    
    for iname in images:
        
        try:
            # Open the image
            img = cv2.imread(iname)
            # Grayscale the image
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Find aruco markers in the query image
            corners, ids, _ = aruco.detectMarkers(
                    image=gray,
                    dictionary=ARUCO_DICT)

            # Outline the aruco markers found in our query image
            img = aruco.drawDetectedMarkers(
                    image=img, 
                    corners=corners)

            # Get charuco corners and ids from detected aruco markers
            response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
                    markerCorners=corners,
                    markerIds=ids,
                    image=gray,
                    board=CHARUCO_BOARD)
            
            # If a Charuco board was found, let's collect image/corner points
            # Requiring at least 20 squares
            if response > 3:
                # Add these corners and ids to our calibration arrays
                corners_all.append(charuco_corners)
                ids_all.append(charuco_ids)

                # Draw the Charuco board we've detected to show our calibrator the board was properly detected
                img = aruco.drawDetectedCornersCharuco(
                        image=img,
                        charucoCorners=charuco_corners,
                        charucoIds=charuco_ids)
                
                # print("charuco_corners:", charuco_corners)
                # print("charuco_ids:", charuco_ids)
                
                # If our image size is unknown, set it now
                if not image_size:
                    image_size = gray.shape[::-1]

                # Reproportion the image, maxing width or height at 1000
                proportion = max(img.shape) / 1000.0
                img = cv2.resize(img, (int(img.shape[1]/proportion), int(img.shape[0]/proportion)))

                # save img to path
                # path = iname[:62]
                fname = os.path.basename(iname)
                
                # print(path)
                
                # print(fname)
                # print(path)
                # print(os.path.join(path, "annotated", fname[:-4]+"-annotated.png"))
                cv2.imwrite(os.path.join(path, "annotated", fname[:-4]+"-annotated.png"), img)

                # Pause to display each image, waiting for key press
                # cv2.imshow('Charuco board', img)
                # cv2.waitKey(0)
            else:
                corners_all.append([])
                ids_all.append([])
#                 print("Not able to detect a charuco board in image: {}".format(iname))
#                 shutil.copy(iname, os.path.join(path, "annotated", "no-detection"))
        except:
            corners_all.append([])
            ids_all.append([])
#             path = iname[:-12]
#             print("Error raised for image {}".format(iname))
#             shutil.copy(iname, os.path.join(path, "annotated", "error-raised"))
        
        # path = iname[:62]
        fname = os.path.basename(iname)
        # print(fname)
    
        img_corners.append(corners_all)
        img_ids.append(ids_all)
        
        # print("corners_all:", corners_all)
        # print("ids_all:", ids_all)
        
    return img_corners[0], img_ids[0]

In [8]:
def csv_rows(path, images, corners_all, ids_all):
    # x = [corners_all[i] for i in range(0, len(corners_all))]
    
    x = []
    for i in range(0, len(corners_all)):
        temp_ids = []
        for j in range(0, len(corners_all[i])):
            temp_ids.append(corners_all[i][j][0][0])
        x.append(temp_ids)
    
    # print("x:", x[0])
    
    # y = [corners_all[-1][i][0][1] for i in range(0, len(corners_all[-1]))]
    
    y = []
    for i in range(0, len(corners_all)):
        temp_ids = []
        for j in range(0, len(corners_all[i])):
            temp_ids.append(corners_all[i][j][0][1])
        y.append(temp_ids)
    
    # print("y:", y[0])
    
    scorer_header = ["DLC_resnet50_Rightonly_FMHMay22shuffle1_750000"]*76
    scorer_header[0]='scorer'
    
    header1 = ['bodyparts']
    for i in range(0, 25):
        header1.append(str(i))
        header1.append(str(i))
        header1.append(str(i))
    
    header2 = ['coords']
    for i in range(0, 25):
        header2.append('x')
        header2.append('y')
        header2.append('likelihood')
    
    # data_rows = [[ids_all[-1][i][0], x[i], y[i]] for i in range(0, len(ids_all[-1]))]
    data_rows = []
    for idx in range(0, len(ids_all)):
        data_single_row = [""]*76
        ids_list = ids_all[idx]
        # print("ids_list:", ids_list)
        data_single_row[0] = str(idx)
        for i in range(0, len(ids_list)):
            temp_idx = header1.index(str(ids_list[i][0]))
            data_single_row[temp_idx]=x[idx][i]
            data_single_row[temp_idx+1]=y[idx][i]
            data_single_row[temp_idx+2]=0.9
        
        data_rows.append(data_single_row)
        idx+=1
    
    # print(data_rows)
    
    return scorer_header, header1, header2, data_rows
        
        
def write_csv(path, scorer_header, header1, header2, data_rows):
    with open(os.path.join(path, 'id_data.csv'), 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(scorer_header)
        writer.writerow(header1)
        writer.writerow(header2)
        writer.writerows(data_rows)

def format_csv(path, images):
    
    img_corners, img_ids = analyze_images(path, images)
    
    # print("img_corners:", img_corners)
    # print("img_ids:", img_ids)
    # print("img_corners:", len(img_corners))
    # print("img_ids:", len(img_ids))
    
    scorer_header, header1, header2, data_rows = csv_rows(path, images, img_corners, img_ids)
    
    write_csv(path, scorer_header, header1, header2, data_rows)

In [17]:
# path = "C:\\Users\\bidayelab\\Documents\\chArUco-detection\\06072022-hind specific\\10\\t2\\camC"
# path = "C:\\Users\\bidayelab\\Documents\\chArUco-detection\\06102022\\set1\\C"
path = "C:\\Users\\bidayelab\\Documents\\chArUco-detection\\new board- set1\\H"

# images = glob.glob('C:\\Users\\bidayelab\\Documents\\chArUco-detection\\06102022\set1\C\*.png')
images = glob.glob('C:\\Users\\bidayelab\\Documents\\chArUco-detection\\new board- set1\\H\\*.png')
images = natsort.natsorted(images)

format_csv(path, images)

In [ ]:
# Make sure at least one image was found
if len(images) < 1:
    # Calibration failed because there were no images, warn the user
    print("Calibration was unsuccessful. No images of charucoboards were found. Add images of charucoboards and use or alter the naming conventions used in this file.")
    # Exit for failure
    exit()

# Make sure we were able to calibrate on at least one charucoboard by checking
# if we ever determined the image size
if not image_size:
    # Calibration failed because we didn't see any charucoboards of the PatternSize used
    print("Calibration was unsuccessful. We couldn't detect charucoboards in any of the images supplied. Try changing the patternSize passed into Charucoboard_create(), or try different pictures of charucoboards.")
    # Exit for failure
    exit()

In [ ]:
# Now that we've seen all of our images, perform the camera calibration
# based on the set of points we've discovered
calibration, cameraMatrix, distCoeffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_all,
        charucoIds=ids_all,
        board=CHARUCO_BOARD,
        imageSize=image_size,
        cameraMatrix=None,
        distCoeffs=None)

In [ ]:
# Print matrix and distortion coefficient to the console
print(cameraMatrix)
print(distCoeffs)

In [ ]:
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_50)
board = aruco.CharucoBoard_create(6, 6, 0.5, 0.375, aruco_dict)
imboard = board.draw((2000, 2000))
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.imshow(imboard, cmap = mpl.cm.gray, interpolation = "nearest")
ax.axis("off")
plt.show()